# CH 7

## TOC<a id='toc'></a>
* [Ch7 Notes](#ch7_notes)

### CH7 Notes <a id='ch7_notes'></a>
[toc](#toc)

## Decorators 101
* a **decorator** is a callable that thakes anotehr function as argument (the *decorated function*)
    - it either processes the function and returns it, or replaces it with another callalbe
    - the @ notation is syntactic sugar - can always simply call a decorator like any regualr callable
* They are executed immediately when the module is loaded
    - that is, they run right after decorated function is defined
* One very useful use case - `@register`. Setup a registry, and register functions/classes.

## Variable Scope rules

In [6]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [7]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [8]:
b

6

* This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local
* When python compiles the body of the function, it decides that b is a local variable because it is assigned in the function.
* use keyword `global` to tell python it is a global variable



In [9]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [10]:
f3(3)

3
6


In [11]:
b

9

## Closures
* DEF: a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.
* Only matter when you have nested functions
    - ofteh the inner functions are anonymous, so people confuse concept of anonymous func with closure.
* **free variable** - a variable that is not bound in the local scope
    - closures have free variables that functions can use
    - these can be found in `myfunc.__code__`, namely in the `.co_freevars` property
    - and the bindings for these free vars are contained in `myfunc.__closure__`
* if free variable is immutable, and you want to do something like += inside the closure, must use `nonlocal` keyword. Otherwise it will think it is local variable and you are trying to "reference it before assigning to it"
    - remember the error above with the print before the assigment!

In [1]:
def myfunc1(x):
    y = x+1
    return y

In [6]:
myfunc1.__code__.co_freevars, myfunc1.__closure__

((), None)

In [39]:
def higher_myfunc():
    counter = 0
    
    def myfunc(x):
        nonlocal counter
        counter = counter + 1
        y = x+1
        return y
    
    return myfunc

In [40]:
lower1 = higher_myfunc()

In [42]:
lower1.__closure__, lower1.__closure__[0].cell_contents

((<cell at 0x00000268773DBEE8: int object at 0x00000000716B7270>,), 0)

In [43]:
for _ in range(10):
    lower1(3)

In [44]:
lower1.__closure__, lower1.__closure__[0].cell_contents

((<cell at 0x00000268773DBEE8: int object at 0x00000000716B73B0>,), 10)

In [45]:
lower2 = higher_myfunc()

In [46]:
lower2.__closure__[0].cell_contents

0

## Decorator
* A decorator is basically a closure where the decorated function is a free variable
* use `functools.wraps` decorator to copy the relevant attributes from decorated function to decorator
    - ex: 

In [49]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, *kwargs)
        elapsed = time.time() - t0
        print('elapsed time: {}'.format(elapsed))
        return result
    
    return clocked

In [54]:
@clock
def myfunc(x):
    "dumb func"
    return x+1

In [55]:
myfunc(3)

elapsed time: 0.0


4

In [56]:
myfunc.__name__

'myfunc'

In [57]:
myfunc.__doc__

'dumb func'

* decorators in the standard library:
    - builtins for decorating methods: `@property`, `@staticmethod`, `@classmethod`
    - memoization with  **Least Recently Used (LRU) cache** `\@functools.lru_cache`
        * uses dictionary - keys are positional and keyword arguments - they must be hashable
        * this one accepts arguments: `maxsize` (determines how many call results are stored) and `typed` (if true, stores results of different typed arguments separately, like 1.0 vs 1 wch are usually considered equal)
        * for optimal performance, maxsize should be a power of 2
    - `\@functools.singledispatch` - if you decorate a plain function with singledispatch, it becomes a **generic function**: a group of functions to perfomr the same operation in different ways depending on the type of th efirst argument.

In [1]:
from functools import singledispatch
from collections import abc
import numbers, html

In [8]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>'+inner+'</li>\n</ul>'


In [10]:
htmlize([3,'3'])

'<ul>\n<li><pre>3</pre></li>\n<li><p>3</p></li>\n</ul>'

* when possible, register the specialized functions to hande ABCs instead of concrete implementations
* notable quality of single dispatch - you can register specialized functions anywhere in the system, in a ny module

<br>
<hr>
Single dipatch is awesome - better than bunch of if/else. But main advantage is really *modular extension*. Each module can register a specialized function for each type it supports.
<hr>
<br>

* stacked decorators apply from the inside out
* to create parameterized decorators:
    - write a *decorator factory* that returns a decorator (basically use a closure)
    - use the paramaterized factory to decorate functions
    - example:

In [23]:
registry = set()

def register(active=True):
    def decorate(func):
        if active:
            registry.add(func)
        return func
    return decorate


@register(active=True)
def f1():
    "Hi"
    print('running f1()')

In [26]:
f1.__doc__

'Hi'

In [27]:
f1()

running f1()


* more realistic examples actually modify the decorated function